In [1]:
# Sample program for one scrip
from ib_insync import *
util.startLoop()
ib=IB().connect('127.0.0.1', 7496, clientId=9)

In [39]:
import numpy as np
import pandas as pd

#... prepare lot dataframe for underlying
# from 5paisa
paisaurl = "https://www.5paisa.com/5pit/spma.asp"
df_paisa = pd.read_html(paisaurl, header=0)[1].drop_duplicates(subset='Symbol')

# Rename Symbol and Margin fields
df_paisa = df_paisa.rename(columns={'Symbol': 'nseSymbol', 'TotMgn%': 'marginpct'})

# Convert columns to numeric and make margin to pct
df_paisa = df_paisa.apply(pd.to_numeric, errors='ignore')
df_paisa.marginpct = df_paisa.marginpct.div(100)

# Truncate to 9 characters for ibSymbol
df_paisa['ibSymbol'] = df_paisa.nseSymbol.str.slice(0,9)

# nseSymbol to ibSymbol dictionary for conversion
ntoi = {'M&M': 'MM', 'M&MFIN': 'MM', 'L&TFH': 'LTFH', 'NIFTY': 'NIFTY50'}

# remap ibSymbol, based on the dictionary
df_paisa.ibSymbol = df_paisa.ibSymbol.replace(ntoi)

#... Get the scrip
symbol = 'ACC'
contract = Stock('ACC', 'NSE')
ib.qualifyContracts(contract)

#... Get the price from bar data 
# Last second's bar of historical with realtime updates
undPrice = ib.reqHistoricalData(contract=contract, endDateTime='', 
                     durationStr='60 S', barSizeSetting='1 secs', 
                     whatToShow='TRADES', useRTH=True, 
                     formatDate=1, keepUpToDate=True)[-1].high

#... Get the volatility
duration = '12 M'
size = '1 day'
bars = ib.reqHistoricalData(contract=contract, endDateTime='', 
                     durationStr=duration, barSizeSetting=size, 
                     whatToShow='MIDPOINT', useRTH=True, 
                     formatDate=1, keepUpToDate=True)
stDev = np.std(a=[b.high for b in bars], ddof=0)

#... Get the lot
lot = df_paisa.loc[df_paisa.ibSymbol == symbol, 'Mlot'].item()

In [42]:
#... Get the option chain
ib.reqSecDefOptParams(underlyingSymbol=contract.symbol, 
                      futFopExchange='', 
                      underlyingConId=contract.conId, underlyingSecType=contract.secType)

[OptionChain(exchange='NSE', underlyingConId=44652144, tradingClass='ACC', multiplier='1', expirations={'20190228', '20190131', '20181227'}, strikes={1280.0, 1920.0, 1540.0, 1160.0, 1800.0, 1420.0, 1040.0, 1680.0, 1300.0, 1940.0, 1560.0, 1180.0, 1820.0, 1440.0, 1660.0, 1060.0, 1700.0, 1320.0, 1960.0, 940.0, 1580.0, 1200.0, 1840.0, 1460.0, 1080.0, 1720.0, 1340.0, 1980.0, 960.0, 1600.0, 1220.0, 1860.0, 1480.0, 1100.0, 1740.0, 1360.0, 980.0, 1620.0, 1240.0, 1880.0, 1500.0, 1120.0, 1760.0, 1380.0, 1000.0, 1640.0, 1260.0, 1900.0, 1520.0, 1140.0, 1780.0, 1400.0, 1020.0})]

ERROR:ib_insync.wrapper:Error 1100, reqId -1: Connectivity between IB and Trader Workstation has been lost.
ERROR:ib_insync.wrapper:Error 1102, reqId -1: Connectivity between IB and Trader Workstation has been restored - data maintained.
ERROR:ib_insync.wrapper:Error 10182, reqId 3570: Failed to request live updates (disconnected)
ERROR:ib_insync.wrapper:Error 10182, reqId 3566: Failed to request live updates (disconnected)
ERROR:ib_insync.wrapper:Error 10182, reqId 3564: Failed to request live updates (disconnected)
ERROR:ib_insync.wrapper:Error 10182, reqId 3565: Failed to request live updates (disconnected)
ERROR:ib_insync.wrapper:Error 10182, reqId 3571: Failed to request live updates (disconnected)
ERROR:ib_insync.wrapper:Error 10182, reqId 3563: Failed to request live updates (disconnected)


In [38]:
contract.conId

0